# Part 1: Scrapping data

In [123]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

1 - Get the data to scrapp using requests and BeautifulSoup libraries. The raw data is stores in the html_data variable

In [124]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
html_data = BeautifulSoup(extracting_data, 'lxml')

html_data

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"e98032b1-784f-4703-a87f-8852783538aa","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-relat

2 - Extract the data. The raw data is processed to a pandas dataframe

In [125]:
col_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = col_names)

content = html_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood}, ignore_index=True)
    
df.describe()

,Postalcode,Borough,Neighborhood
count,181,181,181
unique,181,12,101
top,M6A,Not assigned,Not assigned
freq,1,77,77


3 - Clean the data. Any rows with a non assigned borough is removed as well the first row which has all values as "0". If a Neighborhood is not assigned, the Neighborhood will be the same as the Borough

In [126]:
# Removing cells with a Not assigned Borough and the first row wich have all values as "0"
df = df[df.Borough != 'Not assigned']
df = df[df.Borough != 0]
df.reset_index(drop = True, inplace = True)

# If a Neighborhood is not assigned, the Neighborhood will be the same as the Borough
i = 0
for i in range(0, df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1

     
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [127]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6A,North York,Downsview
freq,1,24,4


In [128]:
df.shape

(103, 3)

# Part 2: Get coordinate of each Neighborhood

1 - Install geocoder

In [51]:
!pip install geocoder

     |████████████████████████████████| 102kB 9.8MB/s eta 0:00:01


In [129]:
import numpy as np
import geocoder

2 - Define a function to get the coordinates and then, get all the coordinates based on the postal codes

In [130]:
def get_coords(code):
    lat_lon = None
    while(lat_lon is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lon = g.latlng
    return lat_lon

In [58]:
postal_codes = df['Postalcode']
coords = []
for code in postal_codes:
    coords.append( get_coords(code) )
coords

[[43.81153000000006, -79.19551999999999],
 [43.78564000000006, -79.15870999999999],
 [43.765750000000025, -79.17519999999996],
 [43.768200000000036, -79.21760999999998],
 [43.769690000000026, -79.23943999999995],
 [43.74309000000005, -79.23525999999998],
 [43.72861000000006, -79.26366999999993],
 [43.714060000000075, -79.28411999999997],
 [43.72360000000003, -79.23495999999994],
 [43.69539000000003, -79.26193999999998],
 [43.75998000000004, -79.26836999999995],
 [43.750710000000026, -79.30055999999996],
 [43.79394000000008, -79.26710999999995],
 [43.784730000000025, -79.29936999999995],
 [43.817810000000065, -79.28023999999994],
 [43.80052000000006, -79.32073999999994],
 [43.83422000000007, -79.21669999999995],
 [43.802850000000035, -79.35620999999998],
 [43.780970000000025, -79.34780999999998],
 [43.78102000000007, -79.38059999999996],
 [43.757220000000075, -79.37973999999997],
 [43.79135000000008, -79.41355999999996],
 [43.76714000000004, -79.40706999999998],
 [43.747870000000034, -7

In [131]:
len( coords )

103

3 - Add the coordinates data to the main dataframe

In [132]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


# Part 3: Exploring Neighborhoods

In [61]:
!pip install geopy
!pip install folium

     |████████████████████████████████| 102kB 11.2MB/s ta 0:00:01


In [133]:
import folium 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


1 - Get the Toronto lat and lon coordinates, then a map of Toronto is created, with the Neighborhoods on top of it

In [134]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto: {}, {}.'.format(latitude, longitude))

Coordinates of Toronto: 43.6534817, -79.3839347.


In [150]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

2 - Explore the Neighborhoods where the Borough contains the word "Toronto"

In [156]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)

(39, 5)


In [157]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

3 - Connecting to the Foursquare API

In [ ]:
# REPLACE THE VARIABLES WITH YOUR OWN CREDENTIALS

CLIENT_ID = 'client-ID'
CLIENT_SECRET = 'client-secret'
VERSION = '20180605'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

4 - Define a funcion to get nearby venues

In [138]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [139]:
toronto_venues = getNearbyVenues(names = df['Neighborhood'], latitudes = df['Latitude'], longitudes = df['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [140]:
print(toronto_venues.shape)
toronto_venues.head(20)

(1376, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.81153,-79.19552,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
4,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Heron Park,43.769327,-79.177201,Park
5,Woburn,43.76820,-79.21761,Starbucks,43.770037,-79.221156,Coffee Shop
6,Woburn,43.76820,-79.21761,cheapOseo,43.766042,-79.218539,Business Service
7,Woburn,43.76820,-79.21761,Densgrove Park,43.765397,-79.220130,Park
8,Woburn,43.76820,-79.21761,Korean Grill House,43.770812,-79.214502,Korean Restaurant
9,Cedarbrae,43.76969,-79.23944,The Avenue,43.772437,-79.241962,Lounge


In [141]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",3,3,3,3,3,3
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",30,30,30,30,30,30


In [142]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


5 - Analyze each Neighborhood

In [143]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Fix the Neighborhood column and put it at the start of the dataframe
toronto_onehot = toronto_onehot.drop('Neighborhood', 1)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

print(toronto_onehot.shape)
toronto_onehot.head()

(1376, 232)


,Neighborhood,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

6 - Group by Neighborhood and get the mean of the frecuency of each category and get the 5 more frecuent of each Neighborhood

In [155]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

(98, 232)

In [145]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Agincourt ----
              venue  freq
0              Park  0.12
1    Discount Store  0.12
2       Supermarket  0.12
3  Sushi Restaurant  0.12
4      Skating Rink  0.12


---- Alderwood, Long Branch ----
               venue  freq
0  Convenience Store  0.17
1                Gym  0.17
2                Pub  0.17
3     Sandwich Place  0.17
4        Pizza Place  0.17


---- Bathurst Manor, Wilson Heights, Downsview North ----
               venue  freq
0               Park  0.33
1        IT Services  0.33
2        Music Venue  0.33
3      Movie Theater  0.00
4  Martial Arts Dojo  0.00


---- Bayview Village ----
                        venue  freq
0  Construction & Landscaping  0.25
1                       Trail  0.25
2                     Dog Run  0.25
3                        Park  0.25
4             Organic Grocery  0.00


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0  Italian Restaurant  0.14
1      Sandwich Place  0.10
2         Coffee Shop  0.10
3  

           venue  freq
0    Coffee Shop  0.17
1   Hockey Arena  0.17
2           Park  0.17
3          Field  0.17
4  Grocery Store  0.17


---- India Bazaar, The Beaches West ----
                  venue  freq
0                  Park  0.10
1  Fast Food Restaurant  0.10
2           Pizza Place  0.10
3             Pet Store  0.05
4           Coffee Shop  0.05


---- Islington Avenue, Humber Valley Village ----
           venue  freq
0       Pharmacy  0.25
1  Shopping Mall  0.12
2           Café  0.12
3           Park  0.12
4   Skating Rink  0.12


---- Kennedy Park, Ionview, East Birchmount Park ----
                venue  freq
0         Coffee Shop   0.2
1          Hobby Shop   0.1
2  Light Rail Station   0.1
3        Hockey Arena   0.1
4    Department Store   0.1


---- Kensington Market, Chinatown, Grange Park ----
                   venue  freq
0                   Café  0.10
1            Pizza Place  0.07
2            Art Gallery  0.07
3  Vietnamese Restaurant  0.07
4     Mexican Re

                venue  freq
0  Italian Restaurant  0.11
1      Sandwich Place  0.07
2                Park  0.07
3                Café  0.07
4  Mexican Restaurant  0.07


---- The Beaches ----
               venue  freq
0  Health Food Store  0.25
1              Trail  0.25
2                Pub  0.25
3            Airport  0.00
4      Movie Theater  0.00


---- The Danforth West, Riverdale ----
           venue  freq
0           Park  0.14
1  Grocery Store  0.14
2       Bus Line  0.14
3   Intersection  0.14
4    Coffee Shop  0.14


---- The Kingsway, Montgomery Road, Old Mill North ----
               venue  freq
0             Lounge   1.0
1            Airport   0.0
2      Movie Theater   0.0
3  Martial Arts Dojo   0.0
4     Massage Studio   0.0


---- Thorncliffe Park ----
                 venue  freq
0          Yoga Studio   0.1
1  Housing Development   0.1
2          Gas Station   0.1
3         Intersection   0.1
4          Coffee Shop   0.1


---- Toronto Dominion Centre, Design Excha

In [146]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [204]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Sushi Restaurant,Discount Store,Pool,Badminton Court,Supermarket,Park,Skating Rink,Fabric Shop,Event Space
1,"Alderwood, Long Branch",Athletics & Sports,Gym,Pub,Convenience Store,Sandwich Place,Pizza Place,Electronics Store,Doctor's Office,Dog Run,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Music Venue,IT Services,Discount Store,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store
3,Bayview Village,Park,Construction & Landscaping,Trail,Dog Run,Discount Store,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Greek Restaurant,Sushi Restaurant,Pharmacy,Café,Butcher,Pub,Restaurant


7 - Clustering Neighborhoods

In [205]:
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:100]

array([1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 2, 1, 2, 1], dtype=int32)

In [206]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Drop rows with NA values and cast Data Labels to int
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552,1,Home Service,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,1,Bar,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520,2,Park,Gym / Fitness Center,Construction & Landscaping,Discount Store,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store
3,M1G,Scarborough,Woburn,43.76820,-79.21761,2,Park,Coffee Shop,Korean Restaurant,Business Service,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944,1,Lounge,Trail,Playground,Discount Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space


In [208]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Park,Electronics Store,Business Service,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space
5,Central Toronto,0,Park,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Gym / Fitness Center,Art Museum,Doctor's Office,Field,Fast Food Restaurant
6,Central Toronto,0,Park,Garden,Playground,Gym Pool,Greek Restaurant,Department Store,Fabric Shop,Event Space,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,0,Park,Bike Trail,Shop & Service,Tennis Court,Playground,Campground,Event Space,Electronics Store,Discount Store,Donut Shop
23,Central Toronto,0,Park,Discount Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store,Eastern European Restaurant
34,West Toronto,0,Park,Convenience Store,Sandwich Place,Residential Building (Apartment / Condo),Discount Store,Farm,Falafel Restaurant,Fabric Shop,Event Space,Electronics Store
